In [1]:
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data=load_digits()

In [3]:
X=pd.DataFrame(data["data"])

In [4]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [5]:
Y=pd.Series(data["target"])

In [6]:
Ydumm=pd.get_dummies(Y)

In [7]:
X_test, X_train, y_test, y_train=train_test_split(X, Y, test_size=0.2, random_state=666 )

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [109]:
lr=LogisticRegression
dtc=DecisionTreeClassifier
knn=KNeighborsClassifier

In [112]:
models=[lr,dtc, knn]

In [145]:
cs=10**np.linspace(-3,1,5)
grid1={"C":cs}
gs1=GridSearchCV(lr(),grid1,scoring='accuracy', cv=5)
gs1.fit(X_train,y_train)
model1=lr(C=gs1.best_params_["C"])
model1.fit(X_train,y_train)
pred1=model1.predict(X_test)
accuracy_score(y_test,pred1)

0.94084899095337504

In [68]:
grid2={"max_depth":list(range(1,20))}
gs2=GridSearchCV(DecisionTreeClassifier(), grid2, scoring='accuracy')
gs2.fit(X_train, y_train)
gs2.best_params_

{'max_depth': 11}

In [147]:
grid3={'n_neighbors':list(range(1,6)), 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
gs3=GridSearchCV(knn,grid3, scoring='accuracy')
gs3.fit(X_train,y_train)
gs3.best_params_

TypeError: get_params() missing 1 required positional argument: 'self'

In [137]:
for x in models:
    if x==lr:
        grids=grid1
    elif x==dtc:
        grids=grid2
    elif x==knn:
        grids=grid3
    gs=GridSearchCV(x(), grids, scoring='accuracy')
    gs.fit(X_train,y_train)
    model=x(gs.best_params_)
    print(model)
    #model.fit(X_train,y_train)
    #pred=model.predict(X_test)
    #print(accuracy_score(y_test,pred ))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty={'C': 0.01}, random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
DecisionTreeClassifier(class_weight=None, criterion={'max_depth': 18},
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors={'n_neighbors': 1},
           p=2, weights='uniform')


In [148]:
grid3={'n_neighbors':list(range(1,6)), 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}

In [149]:
grid3

{'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
 'n_neighbors': [1, 2, 3, 4, 5]}